<a href="https://colab.research.google.com/github/Sajithpemarathna/Energy-Consumption-Data-in-Germany-1995-2020-/blob/main/Model_Implementation_Attempt_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, install required packages
!pip install xgboost tensorflow scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load data
url = "https://raw.githubusercontent.com/Sajithpemarathna/Energy-consumption-prediction-data/refs/heads/main/Use%20of%20Energy%20of%20Energy%20Carriers.csv"
df = pd.read_csv(url)

# Print first few rows and basic info
print(df.head())
print("\nDataset Info:")
print(df.info())

  Branch Code                              Homogeneous Branches  \
0    CPA08-01  Products of agric., hunting and related services   
1    CPA08-01  Products of agric., hunting and related services   
2    CPA08-01  Products of agric., hunting and related services   
3    CPA08-01  Products of agric., hunting and related services   
4    CPA08-01  Products of agric., hunting and related services   

                                Energy Carriers   1995   1996   1997   1998  \
0              Hard coal and hard coal products    888    946    829   1071   
1  Brown coal (lignite) and brown coal products   1477   1346   1275   1056   
2                                     Crude oil      0      0      0      0   
3                                        Petrol   4580   3610   2976   2897   
4                                  Diesel fuels  94156  94655  94546  94130   

    1999   2000   2001  ...   2011   2012   2013   2014   2015   2016   2017  \
0   1185   1329   1692  ...    360   1080 

In [ ]:
# Melt the dataframe to convert years to a single column
df_melted = pd.melt(df,
                    id_vars=['Branch Code', 'Homogeneous Branches', 'Energy Carriers'],
                    var_name='Year',
                    value_name='Consumption')

# Convert Year to numeric
df_melted['Year'] = pd.to_numeric(df_melted['Year'])

# Sort by Year
df_melted = df_melted.sort_values('Year')

# Check the result
print("Shape after melting:", df_melted.shape)
print("\nFirst few rows:")
print(df_melted.head())

Shape after melting: (14976, 5)

First few rows:
    Branch Code                              Homogeneous Branches  \
0      CPA08-01  Products of agric., hunting and related services   
380    CPA08-46    Wts., except of motor vehicles and motorcycles   
381    CPA08-46    Wts., except of motor vehicles and motorcycles   
382    CPA08-46    Wts., except of motor vehicles and motorcycles   
383    CPA08-46    Wts., except of motor vehicles and motorcycles   

                          Energy Carriers  Year  Consumption  
0        Hard coal and hard coal products  1995          888  
380            Other mineral oil products  1995           77  
381                                 Gases  1995        27465  
382                    Renewable energies  1995          117  
383  Electricity and other energy sources  1995        29387  


In [ ]:
# Create dummy variables for categorical columns
df_encoded = pd.get_dummies(df_melted,
                           columns=['Branch Code', 'Homogeneous Branches', 'Energy Carriers'])

# Add time-based features (optional but can help with seasonality)
df_encoded['Year_sin'] = np.sin(2 * np.pi * df_encoded['Year']/df_encoded['Year'].max())
df_encoded['Year_cos'] = np.cos(2 * np.pi * df_encoded['Year']/df_encoded['Year'].max())

# Split features and target
X = df_encoded.drop(['Consumption', 'Year'], axis=1)
y = df_encoded['Consumption']

# Print feature information
print("Number of features:", X.shape[1])
print("\nFeature names:")
print(X.columns.tolist())

Number of features: 110

Feature names:
['Branch Code_CPA08-01', 'Branch Code_CPA08-02', 'Branch Code_CPA08-03', 'Branch Code_CPA08-05', 'Branch Code_CPA08-06', 'Branch Code_CPA08-07-01', 'Branch Code_CPA08-10-01', 'Branch Code_CPA08-13-01', 'Branch Code_CPA08-16', 'Branch Code_CPA08-17', 'Branch Code_CPA08-18', 'Branch Code_CPA08-19', 'Branch Code_CPA08-20', 'Branch Code_CPA08-21', 'Branch Code_CPA08-22', 'Branch Code_CPA08-23', 'Branch Code_CPA08-24-01', 'Branch Code_CPA08-25', 'Branch Code_CPA08-26', 'Branch Code_CPA08-27', 'Branch Code_CPA08-28', 'Branch Code_CPA08-29', 'Branch Code_CPA08-30', 'Branch Code_CPA08-31-01', 'Branch Code_CPA08-33', 'Branch Code_CPA08-35', 'Branch Code_CPA08-36-01', 'Branch Code_CPA08-37-01', 'Branch Code_CPA08-41-01', 'Branch Code_CPA08-43', 'Branch Code_CPA08-45-01', 'Branch Code_CPA08-46', 'Branch Code_CPA08-47', 'Branch Code_CPA08-49', 'Branch Code_CPA08-50', 'Branch Code_CPA08-51', 'Branch Code_CPA08-52', 'Branch Code_CPA08-53', 'Branch Code_CPA08-I

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Initialize Random Forest for feature selection
rf_selector = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selector.fit(X, y)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_selector.feature_importances_
}).sort_values('importance', ascending=False)

# Print top 15 most important features
print("Top 15 Most Important Features:")
print(feature_importance.head(15))

Top 15 Most Important Features:
                                               feature  importance
98                           Energy Carriers_Crude oil    0.579816
100  Energy Carriers_Electricity and other energy s...    0.054729
61   Homogeneous Branches_Electricity, gas, steam a...    0.048722
56   Homogeneous Branches_Coke and refined petroleu...    0.046125
11                                Branch Code_CPA08-19    0.046079
25                                Branch Code_CPA08-35    0.045571
97   Energy Carriers_Brown coal (lignite) and brown...    0.043582
102   Energy Carriers_Hard coal and hard coal products    0.041466
101                              Energy Carriers_Gases    0.020032
105         Energy Carriers_Other mineral oil products    0.017599
107                 Energy Carriers_Renewable energies    0.015101
108                                           Year_sin    0.012025
109                                           Year_cos    0.011342
96               Energy Carrie

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Print shapes
print("\nTraining set shape:", X_train_scaled.shape)
print("Testing set shape:", X_test_scaled.shape)


Training set shape: (11980, 110)
Testing set shape: (2996, 110)


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Create and train XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=7,
    random_state=42
)

# Train the model
xgb_model.fit(X_train_scaled, y_train)

# Make predictions
xgb_pred = xgb_model.predict(X_test_scaled)

# Calculate metrics
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred) * 100

print("XGBoost Metrics:")
print(f"MSE: {xgb_mse:.2f}")
print(f"RMSE: {xgb_rmse:.2f}")
print(f"MAPE: {xgb_mape:.2f}%")

XGBoost Metrics:
MSE: 375108288.00
RMSE: 19367.71
MAPE: 283448957423792947200.00%


In [ ]:
# Create sequences for LSTM/Bi-GRU
def create_sequences(X, y, sequence_length=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:(i + sequence_length)])
        y_seq.append(y[i + sequence_length])
    return np.array(X_seq), np.array(y_seq)

# Create sequences
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, sequence_length)

print("Sequence shapes:")
print("X_train_seq shape:", X_train_seq.shape)
print("X_test_seq shape:", X_test_seq.shape)

KeyError: 6

In [ ]:
# First, let's reset our preprocessing steps
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 1. Ensure data is properly sorted by year
df_encoded = df_encoded.sort_values('Year')

# 2. Scale the features and target separately
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# 3. Prepare the data differently for neural networks
def prepare_time_series_data(X, y, sequence_length=5):
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    X_sequences = []
    y_sequences = []

    for i in range(len(X) - sequence_length):
        X_sequences.append(X_scaled[i:(i + sequence_length)])
        y_sequences.append(y_scaled[i + sequence_length])

    return np.array(X_sequences), np.array(y_sequences)

# 4. Split data first, then create sequences
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=False  # Important: don't shuffle for time series
)

# 5. Create sequences for training and test sets
sequence_length = 5
X_train_seq, y_train_seq = prepare_time_series_data(X_train, y_train, sequence_length)
X_test_seq, y_test_seq = prepare_time_series_data(X_test, y_test, sequence_length)

# Print shapes to verify
print("Training sequences shape:", X_train_seq.shape)
print("Training targets shape:", y_train_seq.shape)
print("Test sequences shape:", X_test_seq.shape)
print("Test targets shape:", y_test_seq.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile LSTM model
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train LSTM model
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate LSTM model
lstm_pred = lstm_model.predict(X_test_seq)
lstm_pred = scaler_y.inverse_transform(lstm_pred)
y_test_actual = scaler_y.inverse_transform(y_test_seq)

# Calculate metrics for LSTM
lstm_mse = mean_squared_error(y_test_actual, lstm_pred)
lstm_rmse = np.sqrt(lstm_mse)
lstm_mape = mean_absolute_percentage_error(y_test_actual, lstm_pred) * 100

print("\nLSTM Model Metrics:")
print(f"MSE: {lstm_mse:.2f}")
print(f"RMSE: {lstm_rmse:.2f}")
print(f"MAPE: {lstm_mape:.2f}%")

In [ ]:
from tensorflow.keras.layers import Bidirectional, GRU

# Build Bi-GRU model with adjusted architecture
bigru_model = Sequential([
    Bidirectional(GRU(128, return_sequences=True), input_shape=(sequence_length, X_train.shape[1])),
    Dropout(0.3),
    Bidirectional(GRU(64)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile with adjusted learning rate
bigru_model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='mse',
    metrics=['mae']
)

# Train with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train Bi-GRU model
bigru_history = bigru_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate Bi-GRU model
bigru_pred = bigru_model.predict(X_test_seq)
bigru_pred = scaler_y.inverse_transform(bigru_pred)

# Calculate metrics for Bi-GRU
bigru_mse = mean_squared_error(y_test_actual, bigru_pred)
bigru_rmse = np.sqrt(bigru_mse)
bigru_mape = mean_absolute_percentage_error(y_test_actual, bigru_pred) * 100

print("\nBi-GRU Model Metrics:")
print(f"MSE: {bigru_mse:.2f}")
print(f"RMSE: {bigru_rmse:.2f}")
print(f"MAPE: {bigru_mape:.2f}%")

In [ ]:
from sklearn.preprocessing import StandardScaler

# Reset our preprocessing steps
scaler_X = StandardScaler()
scaler_y = StandardScaler()

def prepare_time_series_data_v2(X, y, sequence_length=5):
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    # Add a rolling mean feature
    X_df = pd.DataFrame(X_scaled)
    X_df['rolling_mean'] = pd.DataFrame(y).rolling(window=sequence_length).mean()
    X_df = X_df.fillna(method='bfill')

    X_sequences = []
    y_sequences = []

    for i in range(len(X_df) - sequence_length):
        X_sequences.append(X_df.iloc[i:(i + sequence_length)].values)
        y_sequences.append(y_scaled[i + sequence_length])

    return np.array(X_sequences), np.array(y_sequences)

# Prepare sequences with new method
X_train_seq_v2, y_train_seq_v2 = prepare_time_series_data_v2(X_train, y_train, sequence_length)
X_test_seq_v2, y_test_seq_v2 = prepare_time_series_data_v2(X_test, y_test, sequence_length)

# Build a simpler but more robust LSTM model
lstm_model_v2 = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train_seq_v2.shape[2]), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(1)
])

# Compile with huber loss for robustness against outliers
lstm_model_v2.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',
    metrics=['mae']
)

# Add more callbacks for better training
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
]

# Train the model
history_v2 = lstm_model_v2.fit(
    X_train_seq_v2, y_train_seq_v2,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

# Evaluate
lstm_pred_v2 = lstm_model_v2.predict(X_test_seq_v2)
lstm_pred_v2 = scaler_y.inverse_transform(lstm_pred_v2)
y_test_actual_v2 = scaler_y.inverse_transform(y_test_seq_v2)

# Calculate metrics
lstm_mse_v2 = mean_squared_error(y_test_actual_v2, lstm_pred_v2)
lstm_rmse_v2 = np.sqrt(lstm_mse_v2)
lstm_mape_v2 = mean_absolute_percentage_error(y_test_actual_v2, lstm_pred_v2) * 100

print("\nImproved LSTM Model Metrics:")
print(f"MSE: {lstm_mse_v2:.2f}")
print(f"RMSE: {lstm_rmse_v2:.2f}")
print(f"MAPE: {lstm_mape_v2:.2f}%")

In [ ]:
# 1. First, let's check where the NaN values are coming from
print("NaN values in X_train:", np.isnan(X_train).any())
print("NaN values in y_train:", np.isnan(y_train).any())

# 2. Clean the data and handle NaN values
def clean_and_prepare_data(X, y):
    # Fill NaN values in features with mean
    X_clean = pd.DataFrame(X).fillna(pd.DataFrame(X).mean())

    # Remove any rows where target is NaN
    valid_idx = ~pd.isna(y)
    X_clean = X_clean[valid_idx]
    y_clean = y[valid_idx]

    return X_clean.values, y_clean

X_train_clean, y_train_clean = clean_and_prepare_data(X_train, y_train)
X_test_clean, y_test_clean = clean_and_prepare_data(X_test, y_test)

# 3. Modified sequence preparation function
def prepare_time_series_data_v3(X, y, sequence_length=5):
    # Scale the data
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Create sequences
    X_sequences = []
    y_sequences = []

    for i in range(len(X) - sequence_length):
        X_sequences.append(X_scaled[i:(i + sequence_length)])
        y_sequences.append(y_scaled[i + sequence_length])

    return np.array(X_sequences), np.array(y_sequences)

# 4. Prepare sequences with clean data
X_train_seq, y_train_seq = prepare_time_series_data_v3(X_train_clean, y_train_clean)
X_test_seq, y_test_seq = prepare_time_series_data_v3(X_test_clean, y_test_clean)

# 5. Build and train the model
lstm_model = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train_clean.shape[1])),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Train with early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# 6. Evaluate
lstm_pred = lstm_model.predict(X_test_seq)
lstm_pred = scaler_y.inverse_transform(lstm_pred)
y_test_actual = scaler_y.inverse_transform(y_test_seq)

# Calculate metrics
lstm_mse = mean_squared_error(y_test_actual, lstm_pred)
lstm_rmse = np.sqrt(lstm_mse)
lstm_mape = mean_absolute_percentage_error(y_test_actual, lstm_pred) * 100

print("\nLSTM Model Metrics:")
print(f"MSE: {lstm_mse:.2f}")
print(f"RMSE: {lstm_rmse:.2f}")
print(f"MAPE: {lstm_mape:.2f}%")

In [ ]:
def prepare_time_series_data_v3(X, y, sequence_length=5):
    # Convert inputs to numpy arrays if they're pandas Series/DataFrame
    if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
        X = X.to_numpy()
    if isinstance(y, pd.Series):
        y = y.to_numpy()

    # Reshape y if needed
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)

    # Scale the data
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    # Create sequences
    X_sequences = []
    y_sequences = []

    for i in range(len(X) - sequence_length):
        X_sequences.append(X_scaled[i:(i + sequence_length)])
        y_sequences.append(y_scaled[i + sequence_length])

    return np.array(X_sequences), np.array(y_sequences)

# Clean data function
def clean_and_prepare_data(X, y):
    # Convert to DataFrame if not already
    X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X

    # Fill NaN values in features with mean
    X_clean = X_df.fillna(X_df.mean())

    # Remove any rows where target is NaN
    if isinstance(y, pd.Series):
        valid_idx = ~y.isna()
    else:
        valid_idx = ~pd.isna(y)

    X_clean = X_clean[valid_idx]
    y_clean = y[valid_idx]

    return X_clean, y_clean

# Prepare the data
X_train_clean, y_train_clean = clean_and_prepare_data(X_train, y_train)
X_test_clean, y_test_clean = clean_and_prepare_data(X_test, y_test)

# Create sequences
X_train_seq, y_train_seq = prepare_time_series_data_v3(X_train_clean, y_train_clean)
X_test_seq, y_test_seq = prepare_time_series_data_v3(X_test_clean, y_test_clean)

# Print shapes to verify
print("Training sequence shape:", X_train_seq.shape)
print("Training target shape:", y_train_seq.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test target shape:", y_test_seq.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build LSTM model with improved architecture
lstm_model = Sequential([
    # First LSTM layer with more units since we have 110 features
    LSTM(128, input_shape=(sequence_length, 110), return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),

    # Second LSTM layer
    LSTM(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.2),

    # Dense layers for final predictions
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile with appropriate loss and metrics
lstm_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mape']
)

# Add callbacks for better training
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )
]

# Train the model
history = lstm_model.fit(
    X_train_seq,
    y_train_seq,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# Evaluate the model
y_pred = lstm_model.predict(X_test_seq)

# Inverse transform predictions and actual values
y_pred = scaler_y.inverse_transform(y_pred)
y_test_actual = scaler_y.inverse_transform(y_test_seq)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actual, y_pred)
mape = mean_absolute_percentage_error(y_test_actual, y_pred) * 100

print("\nModel Evaluation Metrics:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# 1. Let's first normalize the target variable using log transformation
def preprocess_data(X, y):
    # Log transform the target (adding 1 to handle zeros)
    y_log = np.log1p(y)

    # Scale features using robust scaler
    from sklearn.preprocessing import RobustScaler
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y_log.reshape(-1, 1))

    return X_scaled, y_scaled, scaler_X, scaler_y

# 2. Prepare sequences with better handling of time dependency
def create_sequences_v2(X, y, sequence_length=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:(i + sequence_length)])
        y_seq.append(y[i + sequence_length])
    return np.array(X_seq), np.array(y_seq)

# 3. Preprocess and prepare data
X_train_scaled, y_train_scaled, scaler_X, scaler_y = preprocess_data(X_train_clean, y_train_clean)
X_test_scaled, y_test_scaled, _, _ = preprocess_data(X_test_clean, y_test_clean)

# 4. Create sequences
X_train_seq, y_train_seq = create_sequences_v2(X_train_scaled, y_train_scaled)
X_test_seq, y_test_seq = create_sequences_v2(X_test_scaled, y_test_scaled)

# 5. Build a simpler model with regularization
from tensorflow.keras.regularizers import l2

model = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train_clean.shape[1]),
         return_sequences=True, kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    LSTM(32, kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1)
])

# 6. Compile with Huber loss for robustness
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',
    metrics=['mae']
)

# 7. Train with callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
]

history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# 8. Evaluate and inverse transform predictions
y_pred = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred)
y_pred = np.expm1(y_pred)  # Inverse of log1p

y_test_actual = scaler_y.inverse_transform(y_test_seq)
y_test_actual = np.expm1(y_test_actual)

# Calculate metrics
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_actual, y_pred) * 100

print("\nImproved Model Metrics:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# 1. Let's first normalize the target variable using log transformation
def preprocess_data(X, y):
    # Log transform the target (adding 1 to handle zeros)
    y_log = np.log1p(y)

    # Scale features using robust scaler
    from sklearn.preprocessing import RobustScaler
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y_log.reshape(-1, 1))

    return X_scaled, y_scaled, scaler_X, scaler_y

# 2. Prepare sequences with better handling of time dependency
def create_sequences_v2(X, y, sequence_length=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:(i + sequence_length)])
        y_seq.append(y[i + sequence_length])
    return np.array(X_seq), np.array(y_seq)

# 3. Preprocess and prepare data
X_train_scaled, y_train_scaled, scaler_X, scaler_y = preprocess_data(X_train_clean, y_train_clean)
X_test_scaled, y_test_scaled, _, _ = preprocess_data(X_test_clean, y_test_clean)

# 4. Create sequences
X_train_seq, y_train_seq = create_sequences_v2(X_train_scaled, y_train_scaled)
X_test_seq, y_test_seq = create_sequences_v2(X_test_scaled, y_test_scaled)

# 5. Build a simpler model with regularization
from tensorflow.keras.regularizers import l2

model = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train_clean.shape[1]),
         return_sequences=True, kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    LSTM(32, kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1)
])

# 6. Compile with Huber loss for robustness
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',
    metrics=['mae']
)

# 7. Train with callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
]

history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# 8. Evaluate and inverse transform predictions
y_pred = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred)
y_pred = np.expm1(y_pred)  # Inverse of log1p

y_test_actual = scaler_y.inverse_transform(y_test_seq)
y_test_actual = np.expm1(y_test_actual)

# Calculate metrics
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_actual, y_pred) * 100

print("\nImproved Model Metrics:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
def preprocess_data(X, y):
    # Convert to numpy arrays first
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values

    # Handle zeros and negatives before log transform
    y_min = np.min(y)
    if y_min <= 0:
        y = y - y_min + 1  # Shift data to make it positive

    # Log transform the target
    y_log = np.log1p(y)

    # Scale features and target
    from sklearn.preprocessing import RobustScaler
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()

    # Ensure correct shapes
    X_reshaped = X.reshape((X.shape[0], -1))  # Flatten if needed
    y_reshaped = y_log.reshape(-1, 1)

    X_scaled = scaler_X.fit_transform(X_reshaped)
    y_scaled = scaler_y.fit_transform(y_reshaped)

    return X_scaled, y_scaled, scaler_X, scaler_y, y_min

# Let's try a simpler sequence creation approach
def create_sequences_v2(X, y, sequence_length=5):
    X_seq = []
    y_seq = []

    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:(i + sequence_length)])
        y_seq.append(y[i + sequence_length])

    return np.array(X_seq), np.array(y_seq).reshape(-1, 1)

# Preprocess data
X_train_scaled, y_train_scaled, scaler_X, scaler_y, y_min = preprocess_data(X_train_clean, y_train_clean)
X_test_scaled, y_test_scaled, _, _, _ = preprocess_data(X_test_clean, y_test_clean)

# Create sequences
X_train_seq, y_train_seq = create_sequences_v2(X_train_scaled, y_train_scaled)
X_test_seq, y_test_seq = create_sequences_v2(X_test_scaled, y_test_scaled)

print("Sequence shapes:")
print("X_train_seq shape:", X_train_seq.shape)
print("y_train_seq shape:", y_train_seq.shape)
print("X_test_seq shape:", X_test_seq.shape)
print("y_test_seq shape:", y_test_seq.shape)

# If shapes look correct, we can proceed with model training

In [ ]:
# Build the final model with an optimized architecture
model = Sequential([
    # First LSTM layer with careful tuning
    LSTM(128, input_shape=(sequence_length, 110),
         return_sequences=True,
         kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),

    # Second LSTM layer
    LSTM(64),
    BatchNormalization(),
    Dropout(0.2),

    # Dense layers with careful size reduction
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1)
])

# Compile with Huber loss for robustness
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',
    metrics=['mae']
)

# Add callbacks for training optimization
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.0001,
        verbose=1
    )
]

# Train the model
history = model.fit(
    X_train_seq,
    y_train_seq,
    validation_split=0.2,
    epochs=150,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Make predictions and inverse transform
y_pred = model.predict(X_test_seq)
y_pred_transformed = scaler_y.inverse_transform(y_pred)
y_test_transformed = scaler_y.inverse_transform(y_test_seq)

# Inverse the log transformation and shift
y_pred_final = np.expm1(y_pred_transformed)
y_test_final = np.expm1(y_test_transformed)

if y_min <= 0:
    y_pred_final = y_pred_final + y_min - 1
    y_test_final = y_test_final + y_min - 1

# Calculate final metrics
mse = mean_squared_error(y_test_final, y_pred_final)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_final, y_pred_final)
mape = mean_absolute_percentage_error(y_test_final, y_pred_final) * 100

print("\nFinal Model Metrics:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build the final model
model = Sequential([
    # First LSTM layer
    LSTM(128,
         input_shape=(sequence_length, 110),
         return_sequences=True,
         kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),

    # Second LSTM layer
    LSTM(64,
         kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.2),

    # Dense layers
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1)
])

# Compile with Huber loss
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',
    metrics=['mae']
)

# Training callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.0001,
        verbose=1
    )
]

# Train the model
history = model.fit(
    X_train_seq,
    y_train_seq,
    validation_split=0.2,
    epochs=150,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Print model summary to verify architecture
model.summary()

In [ ]:
# Make predictions and inverse transform
y_pred = model.predict(X_test_seq)
y_pred_transformed = scaler_y.inverse_transform(y_pred)
y_test_transformed = scaler_y.inverse_transform(y_test_seq)

# Inverse the log transformation
y_pred_final = np.expm1(y_pred_transformed)
y_test_final = np.expm1(y_test_transformed)

if y_min <= 0:
    y_pred_final = y_pred_final + y_min - 1
    y_test_final = y_test_final + y_min - 1

# Calculate final metrics
mse = mean_squared_error(y_test_final, y_pred_final)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_final, y_pred_final)
mape = mean_absolute_percentage_error(y_test_final, y_pred_final) * 100

# Print metrics
print("\nFinal Model Metrics:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

# Visualize predictions vs actual values
plt.figure(figsize=(12, 6))
plt.plot(y_test_final[:100], label='Actual', marker='o', alpha=0.5)
plt.plot(y_pred_final[:100], label='Predicted', marker='x', alpha=0.5)
plt.title('Energy Consumption: Actual vs Predicted (First 100 samples)')
plt.xlabel('Time Steps')
plt.ylabel('Energy Consumption')
plt.legend()
plt.grid(True)
plt.show()

# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Calculate prediction intervals (95% confidence)
from scipy import stats

residuals = y_test_final - y_pred_final
std_dev = np.std(residuals)
confidence_interval = 1.96 * std_dev  # 95% confidence interval

print("\nPrediction Interval (95% confidence):")
print(f"±{confidence_interval:.2f} units")

In [ ]:
import matplotlib.pyplot as plt

# Visualize predictions vs actual values
plt.figure(figsize=(12, 6))
plt.plot(y_test_final[:100], label='Actual', marker='o', alpha=0.5)
plt.plot(y_pred_final[:100], label='Predicted', marker='x', alpha=0.5)
plt.title('Energy Consumption: Actual vs Predicted (First 100 samples)')
plt.xlabel('Time Steps')
plt.ylabel('Energy Consumption')
plt.legend()
plt.grid(True)
plt.show()

# Now let's create future predictions (2021-2030)
def create_future_features(last_sequence, num_predictions, sequence_length=5):
    future_sequences = []
    current_sequence = last_sequence.copy()

    for _ in range(num_predictions):
        # Predict next value
        sequence_reshaped = current_sequence.reshape((1, sequence_length, -1))
        next_pred = model.predict(sequence_reshaped)[0]

        # Update sequence
        current_sequence = np.roll(current_sequence, -1, axis=0)
        current_sequence[-1] = next_pred

        future_sequences.append(next_pred)

    return np.array(future_sequences)

# Get last sequence from training data
last_sequence = X_test_seq[-1]

# Generate predictions for next 10 years (2021-2030)
future_years = 10
future_predictions = create_future_features(last_sequence, future_years)

# Inverse transform predictions
future_pred_transformed = scaler_y.inverse_transform(future_predictions)
future_pred_final = np.expm1(future_pred_transformed)
if y_min <= 0:
    future_pred_final = future_pred_final + y_min - 1

# Create years for x-axis
years = np.arange(2021, 2031)

# Plot future predictions
plt.figure(figsize=(12, 6))
plt.plot(years, future_pred_final, marker='o', linestyle='-', label='Forecasted Values')
plt.fill_between(years,
                future_pred_final - confidence_interval,
                future_pred_final + confidence_interval,
                alpha=0.2, label='95% Confidence Interval')
plt.title('Energy Consumption Forecast (2021-2030)')
plt.xlabel('Year')
plt.ylabel('Energy Consumption')
plt.legend()
plt.grid(True)
plt.show()

# Print forecasted values
print("\nForecasted Energy Consumption (2021-2030):")
for year, pred in zip(years, future_pred_final):
    print(f"Year {year}: {pred[0]:.2f} ±{confidence_interval:.2f} units")

In [ ]:
# First, let's organize our historical data by year
def prepare_historical_data(df_melted):
    # Annual total energy consumption
    annual_total = df_melted.groupby('Year')['Consumption'].sum().reset_index()

    # Energy carrier wise consumption
    carrier_consumption = df_melted.groupby(['Year', 'Energy Carriers'])['Consumption'].sum().reset_index()

    # Sector wise consumption (using Homogeneous Branches)
    sector_consumption = df_melted.groupby(['Year', 'Homogeneous Branches'])['Consumption'].sum().reset_index()

    # Get top 6 sectors by total consumption
    top_sectors = sector_consumption.groupby('Homogeneous Branches')['Consumption'].sum()\
                                  .sort_values(ascending=False).head(6).index

    return annual_total, carrier_consumption, sector_consumption, top_sectors

# Prepare historical data
annual_total, carrier_consumption, sector_consumption, top_sectors = prepare_historical_data(df_melted)

# Create comprehensive visualizations
plt.style.use('seaborn')

# 1. Annual Total Energy Consumption (1995-2030)
plt.figure(figsize=(15, 8))
# Historical data
plt.plot(annual_total['Year'], annual_total['Consumption'],
         marker='o', label='Historical', linewidth=2)
# Forecasted data
plt.plot(years, future_pred_final,
         marker='s', linestyle='--', label='Forecasted', linewidth=2)
plt.fill_between(years,
                future_pred_final.flatten() - confidence_interval,
                future_pred_final.flatten() + confidence_interval,
                alpha=0.2, label='95% Confidence Interval')
plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

# 2. Energy Carrier-wise Consumption
# Get top 5 energy carriers
top_carriers = carrier_consumption.groupby('Energy Carriers')['Consumption'].sum()\
                                .sort_values(ascending=False).head(5).index

plt.figure(figsize=(15, 8))
for carrier in top_carriers:
    carrier_data = carrier_consumption[carrier_consumption['Energy Carriers'] == carrier]
    plt.plot(carrier_data['Year'], carrier_data['Consumption'],
            marker='o', label=carrier, linewidth=2)
plt.title('Top 5 Energy Carriers Consumption Trends (1995-2020)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 3. Top 6 Sectors Energy Consumption
plt.figure(figsize=(15, 8))
for sector in top_sectors:
    sector_data = sector_consumption[sector_consumption['Homogeneous Branches'] == sector]
    plt.plot(sector_data['Year'], sector_data['Consumption'],
            marker='o', label=sector[:30] + '...', linewidth=2)
plt.title('Top 6 Sectors Energy Consumption Trends (1995-2020)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary of Forecasted Total Energy Consumption:")
print("Average annual growth rate (2021-2030):",
      f"{((future_pred_final[-1]/future_pred_final[0])**(1/10) - 1)*100:.2f}%")
print("\nTop Energy Carriers (by total consumption):")
for i, carrier in enumerate(top_carriers, 1):
    print(f"{i}. {carrier}")
print("\nTop Sectors (by total consumption):")
for i, sector in enumerate(top_sectors, 1):
    print(f"{i}. {sector}")

In [ ]:
# First install seaborn if not already installed
!pip install seaborn

import matplotlib.pyplot as plt
import seaborn as sns

# 1. Annual Total Energy Consumption (1995-2030)
plt.figure(figsize=(15, 8))
# Historical data
plt.plot(annual_total['Year'], annual_total['Consumption'],
         marker='o', label='Historical', linewidth=2, color='blue')
# Forecasted data
plt.plot(years, future_pred_final,
         marker='s', linestyle='--', label='Forecasted', linewidth=2, color='red')
plt.fill_between(years,
                future_pred_final.flatten() - confidence_interval,
                future_pred_final.flatten() + confidence_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')
plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

# 2. Energy Carrier-wise Consumption
# Get top 5 energy carriers
top_carriers = carrier_consumption.groupby('Energy Carriers')['Consumption'].sum()\
                                .sort_values(ascending=False).head(5).index

plt.figure(figsize=(15, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange']
for carrier, color in zip(top_carriers, colors):
    carrier_data = carrier_consumption[carrier_consumption['Energy Carriers'] == carrier]
    plt.plot(carrier_data['Year'], carrier_data['Consumption'],
            marker='o', label=carrier, linewidth=2, color=color)
plt.title('Top 5 Energy Carriers Consumption Trends (1995-2020)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Top 6 Sectors Energy Consumption
plt.figure(figsize=(15, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']
for sector, color in zip(top_sectors, colors):
    sector_data = sector_consumption[sector_consumption['Homogeneous Branches'] == sector]
    plt.plot(sector_data['Year'], sector_data['Consumption'],
            marker='o', label=sector[:30] + '...', linewidth=2, color=color)
plt.title('Top 6 Sectors Energy Consumption Trends (1995-2020)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary of Forecasted Total Energy Consumption:")
print("Average annual growth rate (2021-2030):",
      f"{((future_pred_final[-1]/future_pred_final[0])**(1/10) - 1)*100:.2f}%")
print("\nTop Energy Carriers (by total consumption):")
for i, carrier in enumerate(top_carriers, 1):
    print(f"{i}. {carrier}")
print("\nTop Sectors (by total consumption):")
for i, sector in enumerate(top_sectors, 1):
    print(f"{i}. {sector}")

In [ ]:
# Calculate residuals and confidence interval properly
def calculate_confidence_interval(y_true, y_pred, confidence=0.95):
    residuals = y_true - y_pred
    std_dev = np.std(residuals)
    z_score = stats.norm.ppf((1 + confidence) / 2)
    return z_score * std_dev

# 1. Annual Total Energy Consumption (1995-2030)
plt.figure(figsize=(15, 8))

# Historical data
plt.plot(annual_total['Year'], annual_total['Consumption'],
         marker='o', label='Historical', linewidth=2, color='blue')

# Calculate confidence interval
conf_interval = calculate_confidence_interval(y_test_actual, y_pred)

# Forecasted data
future_years = np.arange(2021, 2031)
plt.plot(future_years, future_pred_final,
         marker='s', linestyle='--', label='Forecasted', linewidth=2, color='red')

# Add confidence intervals
plt.fill_between(future_years,
                future_pred_final.flatten() - conf_interval,
                future_pred_final.flatten() + conf_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

# 2. Energy Carrier-wise Consumption with forecasts
plt.figure(figsize=(15, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange']
for carrier, color in zip(top_carriers, colors):
    carrier_data = carrier_consumption[carrier_consumption['Energy Carriers'] == carrier]

    # Historical data
    plt.plot(carrier_data['Year'], carrier_data['Consumption'],
            marker='o', label=f'{carrier} (Historical)', linewidth=2, color=color)

    # Add forecast line (simple trend extrapolation)
    x = carrier_data['Year'].values.reshape(-1, 1)
    y = carrier_data['Consumption'].values
    model = LinearRegression()
    model.fit(x, y)

    # Forecast for 2021-2030
    future_x = np.arange(2021, 2031).reshape(-1, 1)
    future_y = model.predict(future_x)
    plt.plot(future_x, future_y, linestyle='--', color=color,
            label=f'{carrier} (Forecast)')

plt.title('Energy Carriers Consumption and Forecasts (1995-2030)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Top 6 Sectors Energy Consumption with forecasts
plt.figure(figsize=(15, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']
for sector, color in zip(top_sectors, colors):
    sector_data = sector_consumption[sector_consumption['Homogeneous Branches'] == sector]

    # Historical data
    plt.plot(sector_data['Year'], sector_data['Consumption'],
            marker='o', label=f'{sector[:30]}... (Historical)', linewidth=2, color=color)

    # Add forecast line
    x = sector_data['Year'].values.reshape(-1, 1)
    y = sector_data['Consumption'].values
    model = LinearRegression()
    model.fit(x, y)

    # Forecast for 2021-2030
    future_x = np.arange(2021, 2031).reshape(-1, 1)
    future_y = model.predict(future_x)
    plt.plot(future_x, future_y, linestyle='--', color=color,
            label=f'{sector[:30]}... (Forecast)')

plt.title('Top 6 Sectors Energy Consumption and Forecasts (1995-2030)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print trend analysis
print("\nTrend Analysis (2021-2030):")
print("\nEnergy Carriers Trends:")
for carrier in top_carriers:
    carrier_data = carrier_consumption[carrier_consumption['Energy Carriers'] == carrier]
    trend = np.polyfit(carrier_data['Year'], carrier_data['Consumption'], 1)[0]
    print(f"{carrier}: {'Increasing' if trend > 0 else 'Decreasing'} trend")

print("\nSector Trends:")
for sector in top_sectors:
    sector_data = sector_consumption[sector_consumption['Homogeneous Branches'] == sector]
    trend = np.polyfit(sector_data['Year'], sector_data['Consumption'], 1)[0]
    print(f"{sector[:50]}...: {'Increasing' if trend > 0 else 'Decreasing'} trend")

In [ ]:
from scipy import stats
from sklearn.linear_model import LinearRegression

# Recalculate confidence interval
def calculate_confidence_interval(y_true, y_pred, confidence=0.95):
    residuals = y_true - y_pred
    std_dev = np.std(residuals)
    z_score = stats.norm.ppf((1 + confidence) / 2)
    return z_score * std_dev

# 1. Annual Total Energy Consumption (1995-2030)
plt.figure(figsize=(15, 8))

# Historical data
x_historical = annual_total['Year'].values
y_historical = annual_total['Consumption'].values
plt.plot(x_historical, y_historical,
         marker='o', label='Historical (1995-2020)', linewidth=2, color='blue')

# Future years for prediction
x_future = np.arange(2021, 2031)

# Predicted data (reshape for plotting)
y_predicted = future_pred_final.reshape(-1)
plt.plot(x_future, y_predicted,
         marker='s', linestyle='--', label='Predicted (2021-2030)',
         linewidth=2, color='red')

# Add confidence intervals
conf_interval = calculate_confidence_interval(y_test_actual, y_pred)
plt.fill_between(x_future,
                y_predicted - conf_interval,
                y_predicted + conf_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14, pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()

# Add annotations for key points
plt.text(2020, y_historical[-1], f'Last Historical: {y_historical[-1]:.0f}',
         verticalalignment='bottom')
plt.text(2029, y_predicted[-1], f'Final Prediction: {y_predicted[-1]:.0f}',
         verticalalignment='bottom')

plt.show()

# Print prediction summary
print("\nPrediction Summary:")
print(f"Last Historical Value (2020): {y_historical[-1]:.2f}")
print(f"First Prediction (2021): {y_predicted[0]:.2f}")
print(f"Final Prediction (2030): {y_predicted[-1]:.2f}")
print(f"95% Confidence Interval: ±{conf_interval:.2f}")

In [ ]:
# 1. Improved preprocessing with feature engineering
def create_features(df_melted):
    # Create yearly aggregates
    yearly_data = df_melted.groupby('Year')['Consumption'].agg([
        ('total', 'sum'),
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max')
    ]).reset_index()

    # Add trend features
    yearly_data['year_norm'] = (yearly_data['Year'] - yearly_data['Year'].min()) / \
                              (yearly_data['Year'].max() - yearly_data['Year'].min())
    yearly_data['trend'] = np.arange(len(yearly_data))

    # Add cyclical features
    yearly_data['year_sin'] = np.sin(2 * np.pi * yearly_data['year_norm'])
    yearly_data['year_cos'] = np.cos(2 * np.pi * yearly_data['year_norm'])

    return yearly_data

# 2. Improved model architecture
def build_improved_model(input_shape):
    model = Sequential([
        LSTM(128, input_shape=input_shape, return_sequences=True,
             kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        LSTM(64, return_sequences=True,
             kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        LSTM(32, kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.1),

        Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dense(8, activation='relu'),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

    return model

# 3. Prepare data
yearly_features = create_features(df_melted)

# Create sequences
def create_sequences_with_features(data, sequence_length=5):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data.iloc[i:(i + sequence_length)].values)
        y.append(data.iloc[i + sequence_length]['total'])
    return np.array(X), np.array(y)

# Scale features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = yearly_features.drop(['Year', 'total'], axis=1)
y = yearly_features['total']

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Create sequences
sequence_length = 5
X_seq, y_seq = create_sequences_with_features(yearly_features, sequence_length)

# Split data
split_idx = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

# 4. Train improved model
improved_model = build_improved_model((sequence_length, X.shape[1]))

callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
]

history = improved_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# 5. Make predictions
def make_future_predictions(model, last_sequence, n_steps=10):
    predictions = []
    curr_seq = last_sequence.copy()

    for _ in range(n_steps):
        pred = model.predict(curr_seq.reshape(1, sequence_length, -1))[0]
        predictions.append(pred)

        # Update sequence
        new_row = curr_seq[-1].copy()
        new_row[0] = pred  # Update the total value
        curr_seq = np.vstack([curr_seq[1:], new_row])

    return np.array(predictions)

# Get predictions
last_sequence = X_test[-1:]
future_predictions = make_future_predictions(improved_model, last_sequence)
future_predictions = scaler_y.inverse_transform(future_predictions)

# Plot results
plt.figure(figsize=(15, 8))
plt.plot(yearly_features['Year'], yearly_features['total'],
         marker='o', label='Historical', linewidth=2, color='blue')

future_years = np.arange(2021, 2031)
plt.plot(future_years, future_predictions,
         marker='s', linestyle='--', label='Predicted', linewidth=2, color='red')

conf_interval = calculate_confidence_interval(y_test,
                                           improved_model.predict(X_test).flatten())
plt.fill_between(future_years,
                future_predictions.flatten() - conf_interval,
                future_predictions.flatten() + conf_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Print prediction summary
print("\nPrediction Summary:")
print(f"Last Historical Value (2020): {yearly_features['total'].iloc[-1]:,.2f}")
print(f"First Prediction (2021): {future_predictions[0][0]:,.2f}")
print(f"Final Prediction (2030): {future_predictions[-1][0]:,.2f}")
print(f"Predicted Change 2020-2030: {((future_predictions[-1][0]/yearly_features['total'].iloc[-1])-1)*100:.1f}%")

In [ ]:
# 1. First, let's check our data dimensions
def print_data_info(X, y, title="Dataset Info"):
    print(f"\n{title}")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print(f"Number of features: {X.shape[-1]}")

# 2. Create sequences with correct dimensions
def create_sequences_with_features(data, sequence_length=5):
    features = data.drop(['Year', 'total'], axis=1).values
    target = data['total'].values

    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(features[i:(i + sequence_length)])
        y.append(target[i + sequence_length])

    return np.array(X), np.array(y)

# 3. Build model with correct input shape
def build_improved_model(input_shape):
    model = Sequential([
        # First LSTM layer
        LSTM(64, input_shape=input_shape, return_sequences=True,
             kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(32, return_sequences=False,
             kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        # Dense layers
        Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(8, activation='relu'),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

    return model

# 4. Prepare data
sequence_length = 5
yearly_features = create_features(df_melted)

# Create and scale sequences
X_seq, y_seq = create_sequences_with_features(yearly_features, sequence_length)

# Scale features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Reshape for scaling
X_seq_reshaped = X_seq.reshape(-1, X_seq.shape[-1])
X_seq_scaled = scaler_X.fit_transform(X_seq_reshaped)
X_seq_scaled = X_seq_scaled.reshape(X_seq.shape)
y_seq_scaled = scaler_y.fit_transform(y_seq.reshape(-1, 1))

# Split data
split_idx = int(len(X_seq_scaled) * 0.8)
X_train = X_seq_scaled[:split_idx]
X_test = X_seq_scaled[split_idx:]
y_train = y_seq_scaled[:split_idx]
y_test = y_seq_scaled[split_idx:]

# Print shapes to verify
print_data_info(X_train, y_train, "Training Data")
print_data_info(X_test, y_test, "Test Data")

# 5. Build and train model
input_shape = (sequence_length, X_train.shape[2])
improved_model = build_improved_model(input_shape)

# Print model summary
improved_model.summary()

# Train model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
]

history = improved_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

# 6. Make predictions
def make_future_predictions(model, last_sequence, scaler_y, n_steps=10):
    predictions = []
    curr_seq = last_sequence.copy()

    for _ in range(n_steps):
        pred = model.predict(curr_seq.reshape(1, *curr_seq.shape), verbose=0)[0]
        predictions.append(pred)

        # Update sequence
        curr_seq = np.roll(curr_seq, -1, axis=0)
        curr_seq[-1] = pred

    predictions = np.array(predictions)
    return scaler_y.inverse_transform(predictions)

# Get predictions
future_predictions = make_future_predictions(improved_model, X_test[-1:], scaler_y)

# Plot results
plt.figure(figsize=(15, 8))
plt.plot(yearly_features['Year'], yearly_features['total'],
         marker='o', label='Historical', linewidth=2, color='blue')

future_years = np.arange(2021, 2031)
plt.plot(future_years, future_predictions,
         marker='s', linestyle='--', label='Predicted', linewidth=2, color='red')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Print summary
print("\nPrediction Summary:")
print(f"Last Historical Value (2020): {yearly_features['total'].iloc[-1]:,.2f}")
print(f"First Prediction (2021): {future_predictions[0][0]:,.2f}")
print(f"Final Prediction (2030): {future_predictions[-1][0]:,.2f}")

In [ ]:
# Modified prediction function to handle shapes correctly
def make_future_predictions(model, last_sequence, scaler_y, n_steps=10):
    predictions = []
    curr_seq = last_sequence.copy()

    # Remove extra dimensions and ensure correct shape
    curr_seq = curr_seq.reshape(-1, last_sequence.shape[-2], last_sequence.shape[-1])

    for _ in range(n_steps):
        # Make prediction
        pred = model.predict(curr_seq, verbose=0)
        predictions.append(pred[0])

        # Update sequence (shift and add new prediction)
        curr_seq = curr_seq.reshape(-1, last_sequence.shape[-2], last_sequence.shape[-1])
        curr_seq = np.roll(curr_seq, -1, axis=1)
        curr_seq[0, -1] = scaler_X.transform(pred.reshape(1, -1))

    predictions = np.array(predictions)
    return scaler_y.inverse_transform(predictions.reshape(-1, 1))

# Get predictions with corrected function
last_sequence = X_test[-1:]
future_predictions = make_future_predictions(improved_model, last_sequence, scaler_y)

# Plot results with confidence intervals
plt.figure(figsize=(15, 8))

# Historical data
plt.plot(yearly_features['Year'], yearly_features['total'],
         marker='o', label='Historical', linewidth=2, color='blue')

# Future predictions
future_years = np.arange(2021, 2031)
plt.plot(future_years, future_predictions,
         marker='s', linestyle='--', label='Predicted', linewidth=2, color='red')

# Calculate confidence intervals
y_pred_test = improved_model.predict(X_test)
residuals = y_test - y_pred_test
confidence_interval = 1.96 * np.std(residuals)

# Add confidence intervals
plt.fill_between(future_years,
                future_predictions.flatten() - confidence_interval,
                future_predictions.flatten() + confidence_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()

# Add annotations for important points
plt.annotate(f'Last Historical (2020): {yearly_features["total"].iloc[-1]:,.0f}',
             xy=(2020, yearly_features['total'].iloc[-1]),
             xytext=(10, 10), textcoords='offset points')
plt.annotate(f'2030 Prediction: {future_predictions[-1][0]:,.0f}',
             xy=(2030, future_predictions[-1][0]),
             xytext=(10, -10), textcoords='offset points')

plt.tight_layout()
plt.show()

# Print detailed prediction summary
print("\nPrediction Summary:")
print(f"Last Historical Value (2020): {yearly_features['total'].iloc[-1]:,.2f}")
print(f"First Prediction (2021): {future_predictions[0][0]:,.2f}")
print(f"Final Prediction (2030): {future_predictions[-1][0]:,.2f}")
print(f"\nPredicted Change (2020-2030): {((future_predictions[-1][0]/yearly_features['total'].iloc[-1])-1)*100:.1f}%")
print(f"95% Confidence Interval: ±{confidence_interval:.2f}")

In [ ]:
def make_future_predictions(model, last_sequence, scaler_y, n_steps=10):
    predictions = []
    curr_seq = last_sequence.copy()

    # Keep track of the last prediction for feature updates
    last_features = curr_seq[0, -1].copy()

    for _ in range(n_steps):
        # Make prediction with current sequence
        pred = model.predict(curr_seq, verbose=0)
        predictions.append(pred[0])

        # Update features for next prediction
        # Preserving the feature structure of last_features
        next_features = last_features.copy()
        next_features = next_features.reshape(1, -1)

        # Roll the sequence and update last position
        curr_seq = np.roll(curr_seq, -1, axis=1)
        curr_seq[0, -1] = next_features

        # Update last_features for next iteration
        last_features = next_features[0]

    # Convert predictions to numpy array and inverse transform
    predictions = np.array(predictions)
    return scaler_y.inverse_transform(predictions.reshape(-1, 1))

# Let's verify our data shapes before making predictions
print("X_test shape:", X_test.shape)
print("Last sequence shape:", X_test[-1:].shape)
print("Number of features:", X_test.shape[-1])

# Get predictions
last_sequence = X_test[-1:]
future_predictions = make_future_predictions(improved_model, last_sequence, scaler_y)

# Plot results with confidence intervals
plt.figure(figsize=(15, 8))

# Historical data
historical_years = yearly_features['Year']
historical_values = yearly_features['total']
plt.plot(historical_years, historical_values,
         marker='o', label='Historical', linewidth=2, color='blue')

# Future predictions
future_years = np.arange(2021, 2031)
plt.plot(future_years, future_predictions,
         marker='s', linestyle='--', label='Predicted', linewidth=2, color='red')

# Calculate and plot confidence intervals
test_predictions = model.predict(X_test)
test_predictions = scaler_y.inverse_transform(test_predictions)
test_actual = scaler_y.inverse_transform(y_test)
std_error = np.std(test_actual - test_predictions)
confidence_interval = 1.96 * std_error

plt.fill_between(future_years,
                future_predictions.flatten() - confidence_interval,
                future_predictions.flatten() + confidence_interval,
                alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Germany\'s Total Energy Consumption (1995-2030)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Energy Consumption (TJ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()

# Add annotations
plt.annotate(f'Last Historical (2020): {historical_values.iloc[-1]:,.0f}',
             xy=(2020, historical_values.iloc[-1]),
             xytext=(10, 10), textcoords='offset points')
plt.annotate(f'2030 Prediction: {future_predictions[-1][0]:,.0f}',
             xy=(2030, future_predictions[-1][0]),
             xytext=(10, -10), textcoords='offset points')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nPrediction Summary:")
print(f"Last Historical Value (2020): {historical_values.iloc[-1]:,.2f}")
print(f"First Prediction (2021): {future_predictions[0][0]:,.2f}")
print(f"Final Prediction (2030): {future_predictions[-1][0]:,.2f}")
print(f"\nPredicted Change (2020-2030): {((future_predictions[-1][0]/historical_values.iloc[-1])-1)*100:.1f}%")
print(f"95% Confidence Interval: ±{confidence_interval:.2f}")